In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")

workspace = input("Enter full path to your geodatabase: ")

if not arcpy.Exists(workspace):
    raise ValueError("Geodatabase path is not valid.")

env.workspace = workspace
env.overwriteOutput = True

print("Environment ready.")

In [ ]:
# -----------------------------
# DEM Selection
# -----------------------------

rasters = arcpy.ListRasters()

if not rasters:
    raise ValueError("No rasters found in the geodatabase.")

print("\nAvailable DEMs:")
for r in rasters:
    print("-", r)

while True:
    dem = input("\nEnter DEM name from the list above: ")
    if dem in rasters:
        break
    else:
        print("Invalid DEM name. Please choose from the list.")

print(f"Using DEM: {dem}")

In [ ]:
# -----------------------------
# Grid Spacing Selection
# -----------------------------

while True:
    try:
        cell_size = int(input("Enter grid spacing in meters (e.g., 1000 or 2000): "))
        if cell_size <= 0:
            print("Grid spacing must be a positive number.")
            continue
        break
    except ValueError:
        print("Please enter a valid integer value.")

print(f"Grid spacing set to: {cell_size} meters")

In [19]:
desc = arcpy.Describe(dem)
extent = desc.extent

xmin = extent.XMin
ymin = extent.YMin
xmax = extent.XMax
ymax = extent.YMax

origin_coord = f"{xmin} {ymin}"
y_axis_coord = f"{xmin} {ymin + 10}"
corner_coord = f"{xmax} {ymax}"

In [20]:
fishnet_name = f"fishnet_{cell_size}m"
observer_points = f"observer_points_{cell_size}m"

In [21]:
# -----------------------------
# Check if Fishnet already exists
# -----------------------------

if arcpy.Exists(fishnet_name):
    print(f"{fishnet_name} already exists.")
    
    while True:
        choice = input("Do you want to overwrite it? (yes/no): ").lower()
        
        if choice == "yes":
            arcpy.management.Delete(fishnet_name)
            print("Old fishnet deleted. Creating a new one...")
            break
        
        elif choice == "no":
            print("Using existing fishnet.")
            break
        
        else:
            print("Please type 'yes' or 'no'.")

In [ ]:
if not arcpy.Exists(fishnet_name):
    arcpy.management.CreateFishnet(
        out_feature_class=fishnet_name,
        origin_coord=origin_coord,
        y_axis_coord=y_axis_coord,
        cell_width=cell_size,
        cell_height=cell_size,
        corner_coord=corner_coord,
        labels="NO_LABELS",
        geometry_type="POLYGON"
    )

    print(f"{fishnet_name} created successfully.")

In [23]:
observer_points = f"observer_points_{cell_size}m"

In [ ]:
# -----------------------------
# Check if Observer Points already exist
# -----------------------------

if arcpy.Exists(observer_points):
    print(f"{observer_points} already exists.")
    
    while True:
        choice = input("Do you want to overwrite observer points? (yes/no): ").lower()
        
        if choice == "yes":
            arcpy.management.Delete(observer_points)
            print("Old observer points deleted. Creating new ones...")
            break
        
        elif choice == "no":
            print("Using existing observer points.")
            break
        
        else:
            print("Please type 'yes' or 'no'.")

In [ ]:
if not arcpy.Exists(observer_points):
    arcpy.management.FeatureToPoint(
        in_features=fishnet_name,
        out_feature_class=observer_points,
        point_location="INSIDE"
    )

    print(f"{observer_points} created successfully.")

In [ ]:
# -----------------------------
# Multi-Height Input
# -----------------------------

while True:
    try:
        height_input = input("Enter observer heights separated by commas (e.g., 6,15,30): ")
        height_list = [float(h.strip()) for h in height_input.split(",")]
        
        if any(h <= 0 for h in height_list):
            print("All heights must be positive numbers.")
            continue
        
        break
    
    except ValueError:
        print("Please enter valid numeric values separated by commas.")

print("Heights to analyze:", height_list)

In [ ]:
# -----------------------------
# Ensure Obs_Height field exists
# -----------------------------

fields = [f.name for f in arcpy.ListFields(observer_points)]

if "Obs_Height" not in fields:
    arcpy.management.AddField(observer_points, "Obs_Height", "DOUBLE")
    print("Obs_Height field created.")
else:
    print("Obs_Height field already exists.")

In [ ]:
# -----------------------------
# Run Multi-Scenario Viewshed
# -----------------------------

for h in height_list:

    print(f"\nProcessing height: {h} m")

    arcpy.management.CalculateField(
        in_table=observer_points,
        field="Obs_Height",
        expression=str(h),
        expression_type="PYTHON3"
    )

    output_name = f"viewshed_{str(h).replace('.', '_')}m"

    out_viewshed = Viewshed2(
        in_raster=dem,
        in_observer_features=observer_points,
        analysis_type="FREQUENCY",
        vertical_error="0 Meters",
        refractivity_coefficient=0.13,
        surface_offset="Obs_Height"
    )

    out_viewshed.save(output_name)

    print(f"{output_name} created successfully.")

In [ ]:
print("\n-----------------------------------")
print(f"Running viewshed for height: {h} meters")
print("-----------------------------------")

In [ ]:
print(f"Viewshed result saved as: {output_name}")

In [ ]:
# -----------------------------
# Calculate Visibility Statistics
# -----------------------------

total_cells = 0
visible_cells = 0
high_cells = 0

with arcpy.da.SearchCursor(output_name, ["Value", "Count"]) as cursor:
    for value, count in cursor:
        total_cells += count
        
        if value > 0:
            visible_cells += count
        
        if value >= 10:
            high_cells += count

percentage_visible = (visible_cells / total_cells) * 100
high_percentage = (high_cells / total_cells) * 100

print(f"Visible area percentage ({h}m): {percentage_visible:.2f}%")
print(f"High visibility area percentage ({h}m): {high_percentage:.2f}%")

In [ ]:
print("\n===================================")
print("All scenarios completed successfully.")
print("Workspace used:", env.workspace)
print("===================================")